# ⚛️ H₂O — Bond Angle Optimization via Noiseless VQE (UCCSD)

This notebook scans **H–O–H bond angles** and computes the **ground-state energy**
of water using the production VQE API:

```python
from vqe.core import run_vqe_geometry_scan
```

For each bond angle, the package:
- builds the H₂O geometry (fixed O–H bond length, variable H–O–H angle)
- constructs the molecular Hamiltonian in STO-3G
- runs noiseless VQE with a UCCSD ansatz
- reuses cached runs when available
- produces an energy vs bond angle plot

In [ ]:
import numpy as np

from vqe.core import run_vqe_geometry_scan
from vqe.hamiltonian import generate_geometry

## 🧬 Angle grid (degrees)

In [ ]:
angles = np.linspace(100.0, 109.0, 5)
print("H–O–H angle grid (degrees):", angles)

## 🚀 Geometry scan via `run_vqe_geometry_scan`

In [ ]:
results = run_vqe_geometry_scan(
    molecule="H2O_ANGLE",
    param_name="angle",
    param_values=angles,
    ansatz_name="UCCSD",
    optimizer_name="Adam",
    steps=50,
    stepsize=0.2,
    seeds=[0],
    force=False,
    mapping="jordan_wigner",
    show=True,
)

## 📌 Extract approximate equilibrium bond angle

In [ ]:
params, mean_E, std_E = zip(*results)
params = np.array(params, dtype=float)
mean_E = np.array(mean_E, dtype=float)
std_E = np.array(std_E, dtype=float)

min_idx = int(np.argmin(mean_E))
opt_angle = float(params[min_idx])
opt_energy = float(mean_E[min_idx])
opt_sigma = float(std_E[min_idx])

print(f"\nEstimated equilibrium H–O–H bond angle: {opt_angle:.2f}°")
print(f"Minimum mean VQE ground-state energy: {opt_energy:.8f} ± {opt_sigma:.8f} Ha")

## 🔎 Visualize the geometry at the optimal angle (package geometry helper)

In [ ]:
import matplotlib.pyplot as plt
from common.molecule_viz import plot_molecule

sym, coords = generate_geometry("H2O_ANGLE", opt_angle)

plot_molecule(
    sym,
    coords,
    title=f"H₂O geometry at angle ≈ {opt_angle:.2f}°",
    bonds=[(0, 1), (0, 2)],
    angles=[(1, 0, 2)],
    show_bond_lengths=True,
    show_atom_indices=False,
)

plt.show()

---
## Summary

- Used the production VQE API (`run_vqe_geometry_scan`) as a pure client
- Performed a noiseless UCCSD VQE bond-angle scan for H₂O
- Extracted an approximate equilibrium bond angle
- Visualized the corresponding geometry using the package’s geometry generator